<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#load-model" data-toc-modified-id="load-model-1">load model</a></span></li><li><span><a href="#test-model" data-toc-modified-id="test-model-2">test model</a></span></li></ul></div>

# load model

In [1]:
import os
import glob
import cv2
import tensorflow as tf
import numpy as np
import json
from tqdm.notebook import tqdm

In [2]:
work_dir = os.path.join(os.getenv("HOME"), "UDIGO")
model_dir = os.path.join(work_dir, "models")
test_dir = os.path.join(work_dir, "test_data")
data_dir = os.path.join(work_dir, "data")

In [3]:
model_list = os.listdir(model_dir)
model_list

['EfficientNetB0-9-0.04-0.99.h5', 'EfficientNetB1-6-0.02-0.99.h5']

In [4]:
model = tf.keras.models.load_model(os.path.join(model_dir, model_list[1]))

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
efficientnetb1 (Functional)  (None, 1280)              6575239   
_________________________________________________________________
dense (Dense)                (None, 512)               655872    
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 7,238,289
Trainable params: 7,175,210
Non-trainable params: 63,079
_________________________________________________________________


# test model

In [6]:
def test_model(img_path):
    image = cv2.imread(img_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (256, 256))
    image = image[np.newaxis, :, :, :]
    pred = model.predict(image, batch_size=1)
    return np.argmax(pred)

In [7]:
with open(os.path.join(data_dir, "place_10_label.json"), "r") as f:
    label_dict = json.load(f)

In [8]:
image_list = glob.glob(test_dir + "/*/*")
len(image_list)

1939

In [9]:
image_list[:10]

['/home/ssac21/UDIGO/test_data/산/87.jpg',
 '/home/ssac21/UDIGO/test_data/산/183.jpg',
 '/home/ssac21/UDIGO/test_data/산/138.jpg',
 '/home/ssac21/UDIGO/test_data/산/6.jpg',
 '/home/ssac21/UDIGO/test_data/산/178.jpg',
 '/home/ssac21/UDIGO/test_data/산/180.jpg',
 '/home/ssac21/UDIGO/test_data/산/67.jpg',
 '/home/ssac21/UDIGO/test_data/산/192.jpg',
 '/home/ssac21/UDIGO/test_data/산/184.jpg',
 '/home/ssac21/UDIGO/test_data/산/153.jpg']

In [10]:
correct_dict = {}
correct = 0
n = 0

for image in tqdm(image_list):
    try:
        pred = test_model(image)
        n += 1

        label = image.split("/")[-2]
        if label not in correct_dict:
            correct_dict[label] = 0

        label_encoding = label_dict[label]
        if pred == label_encoding:
            correct_dict[label] += 1
            correct += 1
    
    except:
        print("Invalid image:", image)
        continue
        
print(f"{n} data, test complete!")

Invalid image: /home/ssac21/UDIGO/test_data/아쿠아리움/070506553201908240084b19e-97ec-4ee2-b208-2c01c3c3fa1d.gif
Invalid image: /home/ssac21/UDIGO/test_data/공원/30.jpg

1937 data, test complete!


In [11]:
correct_dict

{'산': 200,
 '미술관': 179,
 '호텔': 191,
 '아이스링크': 177,
 '아쿠아리움': 197,
 '다리': 201,
 '공원': 189,
 '공항': 156,
 '놀이동산': 79,
 '볼링장': 196}

In [12]:
accuracy = round(correct / n, 2)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.91
